You find yourself staring at a dataset with tens or hundreds of thousands of rows. Maybe you want to get up-to-date FOIA contact details for all government departments in your country, or to find out which political donors have links to the fossil fuels industry. What do you do?

This kind of work is time-consuming and challenging. However, Large Language Models (LLMs) like those powering ChatGPT can help journalists automate simple research and classification tasks that would take an unreasonably long time to do otherwise.

In this session, we'll outline how to use LLMs, search engines and web scraping to help us identify links between Donald Trump and his donors. You can [download the notebook](https://github.com/nicucalcea/ddj-wiki/blob/main/ai/python-classification-agent.ipynb) and run it yourself, or you can [run it in the cloud using Google Colab](https://colab.research.google.com/github/nicucalcea/ddj-wiki/blob/main/ai/python-classification-agent.ipynb). Both links also in the sidebar to the right, or at the bottom of the page on mobile.

## Install and load libraries

First, we'll need to install some libraries to help us call different LLMs and retrieve search results.

In [31]:
!uv pip install pandas
!uv pip install pydantic-ai
!uv pip install duckduckgo-search
!uv pip install trafilatura

Using Python 3.13.2 environment at: D:\Projects\ddj-wiki\.venv
Audited 1 package in 81ms
Using Python 3.13.2 environment at: D:\Projects\ddj-wiki\.venv
Audited 1 package in 1.03s
Using Python 3.13.2 environment at: D:\Projects\ddj-wiki\.venv
Audited 1 package in 25ms
Using Python 3.13.2 environment at: D:\Projects\ddj-wiki\.venv
Resolved 18 packages in 692ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 10 packages in 1.37s
 + babel==2.17.0
 + courlan==1.3.2
 + dateparser==1.2.1
 + htmldate==1.9.3
 + justext==3.0.2
 + lxml-html-clean==0.4.2
 + regex==2024.11.6
 + tld==0.13
 + trafilatura==2.0.0
 + tzlocal==5.3.1


Then, we'll import those libraries.

In [30]:
# maths
import pandas as pd

# scraping
from urllib import request
import ssl
import requests
from trafilatura import extract

# validation
from typing import List, Tuple
from pydantic import BaseModel, Field
from typing import Literal
import json

# AI
from pydantic_ai import Agent, Tool
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool

# make async work in jupyter
import nest_asyncio
nest_asyncio.apply()

Finally, we need to define some of the API keys we'll be using. You can get your own [OpenAI API key](https://platform.openai.com/account/api-keys) or from another provider.

In [2]:
OPENAI_API_KEY=""

## Prep data

We'll work with some data from the US Federal Election Commission (FEC) on [donations to Donald Trump's inaugural committee](https://docquery.fec.gov/cgi-bin/forms/C00894162/1889684/f132). Because a downloadable version isn't provided, we'll scrape the data directly from the website.

In [2]:
response = request.urlopen("https://docquery.fec.gov/cgi-bin/forms/C00894162/1889684/f132",
                           context=ssl._create_unverified_context())
html = response.read()

fec_raw = pd.read_html(html)[0]

We need to fix some of the formatting issues, like removing dollar signs and converting dates to a more standard format.

In [3]:
fec = fec_raw.copy()
fec = fec.iloc[:-1] # the last row is a summary

fec['Date Donation Received'] = pd.to_datetime(
    fec['Date Donation Received'], 
    format='%m/%d/%Y'  # Format for "31/12/2025"
)

fec['Donation Amount'] = fec['Donation Amount'].str.replace('$', '', regex=False)
fec['Donation Amount'] = pd.to_numeric(fec['Donation Amount'])

fec = fec.drop(columns=["Donor's Aggregate Donations To Date"])

Some donors donated multiple times, so we'll want to group them together and only classify them once.

In [34]:
fec_total = fec.groupby('Name')['Donation Amount'].sum().reset_index()
fec_total = fec_total.sort_values(by='Donation Amount', ascending=False)
fec_total = fec_total.head(5)
fec_total.to_csv('data/fec_unclassified.csv', index=False)

We should be ready to go. Let's see what the data looks like. To keep things simple, we're only going to use the first 5 rows of the dataset.

In [32]:
fec_total

,Name,Donation Amount
618,PILGRIM'S PRIDE CORPORATION,5000000.00
652,"RIPPLE LABS, INC.",4889345.33
824,WARREN A STEPHENS,4000000.00
139,CHEVRON PRODUCTS COMPANY,2000000.00
667,"ROBINHOOD MARKETS, INC.",2000000.00


## Searching and scraping

Large Language Models are prone to [hallucinations](https://en.wikipedia.org/wiki/Hallucination_(artificial_intelligence)). If you ask an LLM a question it doesn't know the answer to, [it will confidently make up a plausible-sounding answer](https://ddj.nicu.md/ai/) that is completely wrong. This is particularly the case with less known organisations or individuals that wouldn't feature promionently in the training data.

One way to minimise (but not completely eliminate) hallucinations is by "grounding" the LLM with some factual data like documents or databases. This is sometimes a technique called Retrieval-Augmented Generation (RAG).

In our case, we want to allow out agent to search the internet to look for additional information about each donor.

We could try simply scraping search results from Google like we did with the FEC data, but Google doesn't like that, and they will put various roadblocks in your way such as CAPTCHAs, rate limits or even blocking your IP address. Instead, they want you to use their [Custom Search API](https://developers.google.com/custom-search/v1/overview), which is paid and limited to 10k queries per day.

Other search engines like Brave provide generous free tiers and more reasonable pricing, and there are some services that specifically cater to AI applications, like [Tavily](https://tavily.com/) and [Perplexity](https://docs.perplexity.ai/home).

We'll use the DuckDuckGo API since it's free (but rate-limited), doesn't require signing up for an API key, and comes with PydanticAI.

We also want to give our AI the ability to visit each of those search results and extract the text from there. We'll use a library called [Trafilatura](https://github.com/adbar/trafilatura) since it extract the main text without headers, footers and other irrelevant bits that we probably don't need for the classification.

In [7]:
def extract_text(urls: List[str]) -> List[Tuple[str, str]]:
    """
    Extract text content from a list of URLs.
    
    Args:
        urls: A list of URLs to scrape
        
    Returns:
        A list of tuples containing (url, extracted_text)
    """
    results = []

    for url in urls:
        # print(f"Scraping {url}...")
        try:
            response = requests.get(url, timeout=30, verify=False)
            response.raise_for_status()
            extracted_text = extract(response.text, output_format="markdown")
            results.append((url, extracted_text))
        except requests.RequestException as e:
            # print(f"Error scraping {url}: {str(e)}")
            results.append((url, f"Error: {str(e)}"))

    return results

## Structured output

Most modern LLMs are trained to produce structured output, which means they can return data in a specific format. This is useful for producing consistent outputs when we're asking the model to classify or research multiple items.

We can define the output format we expect using a Pydantic model.

In [15]:
class TrumpConnection(BaseModel):
    donor_industry: str
    type_of_connection: Literal['Family', 'Business', 'Political', 'Other', 'No connection', 'Don\'t know or not enough information']
    explanation: str
    source_urls: str

## Agents

In an earlier iteration of this project, we searched the internet for each one of those donors using the exact name as listed in the data, extracted the first 10 results, scraped them, and then fed them to an LLM.

Since then, AI agents have become a lot more prominent. Agents are systems where LLMs are allowed to choose their own steps, use tools appropriate for different tasks and make decisions about when to stop.

The code required to set up an agent from scratch is fairly simple, but there are several frameworks which come with batteries included. We'll use [PydanticAI](https://ai.pydantic.dev), but most frameworks provide a similar set of features.

In [9]:
agent = Agent(
    'openai:gpt-4.1-mini',
    tools=[duckduckgo_search_tool(), Tool(extract_text)],
    system_prompt='Answer the question to the best of your abilities, using the tools at your disposal (`duckduckgo_search_tool` and `extract_text`).',
    output_type=TrumpConnection
)

In [51]:
result = agent.run_sync(
    'What is the connection between WARREN A STEPHENS and Donald Trump?'
)

Scraping https://en.wikipedia.org/wiki/Warren_Stephens...
Scraping https://apnews.com/article/trump-transition-appointments-ambassador-52e4744db02a0f8e078685a7a754835d...
Scraping https://www.timesnownews.com/world/us/us-news/who-is-warren-stephens-donald-trump-uk-ambassador-pick-age-net-worth-career-family-controversiesotherdetails-article-115912443...
Error scraping https://www.timesnownews.com/world/us/us-news/who-is-warren-stephens-donald-trump-uk-ambassador-pick-age-net-worth-career-family-controversiesotherdetails-article-115912443: 403 Client Error: Forbidden for url: https://www.timesnownews.com/world/us/us-news/who-is-warren-stephens-donald-trump-uk-ambassador-pick-age-net-worth-career-family-controversiesotherdetails-article-115912443
Scraping https://www.the-express.com/news/us-news/170302/trump-appoints-warren-stephens-us-ambassador-britain-bolstering-uk-us-relations...
Scraping https://www.thv11.com/article/news/local/warren-stephens-confirmed-uk-ambassador/91-7e1fc4e2-3a9

In [52]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Answer the question to the best of your abilities, using the tools at your disposal (`duckduckgo_search_tool` and `extract_text`).', timestamp=datetime.datetime(2025, 5, 19, 12, 30, 38, 290935, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='What is the connection between WARREN A STEPHENS and Donald Trump?', timestamp=datetime.datetime(2025, 5, 19, 12, 30, 38, 290938, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], instructions=None, kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='duckduckgo_search', args='{"query":"WARREN A STEPHENS and Donald Trump connection"}', tool_call_id='call_rorRZr83ZppMbfX2hCOO234n', part_kind='tool-call')], usage=Usage(requests=1, request_tokens=220, response_tokens=26, total_tokens=246, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0}), model_name

In [57]:
result.output

TrumpConnection(type_of_connection='Political', explanation="Warren A Stephens is an American businessman and investment banker who was nominated by former President Donald Trump to serve as the United States ambassador to the United Kingdom in his second administration. Stephens is a significant Republican donor who initially opposed Trump in 2016 but later supported him financially in the 2020 election, donating millions to Trump-aligned Super PACs. Trump announced Stephens' nomination to the ambassador role in December 2024, and the U.S. Senate confirmed it in April 2025. This political appointment and Stephens' financial support to Trump establish the connection between them.", source_urls='https://en.wikipedia.org/wiki/Warren_Stephens')

## Running on the full dataset

We now have a working agent that can search the internet, read websites and return structured outputs.

We can now have it run on each row of our dataset and return a full classification for each donor.

First, let's define a simple function that incorporates the agent run shown above.

In [10]:
def classify_donor(donor: str):
    result = agent.run_sync(f'What is the connection between {donor} and Donald Trump?')
    return result.output

Now, we can run it on the scraped data from the FEC website. In the interest of time, we'll only run it on the first five rows.

In [ ]:
def apply_classify_donor(df):
    # Apply the classify_donor function to each row
    df['classification'] = df.apply(lambda row: classify_donor(donor=row['Name']), axis=1)
    
    # Convert the Pydantic objects to dictionaries
    classification_dicts = df['classification'].apply(lambda x: x.model_dump())
    
    # Create a new DataFrame from the classification dictionaries
    # This ensures the index matches the original dataframe
    classification_df = pd.DataFrame(classification_dicts.tolist(), index=df.index)
    
    # Combine original data with classification data
    result_df = pd.concat([df.drop(['classification'], axis=1), classification_df], axis=1)
    
    return result_df

In [28]:
fec_classified = apply_classify_donor(fec_total)

In [29]:
fec_classified

,Name,Donation Amount,type_of_connection,explanation,source_urls
618,PILGRIM'S PRIDE CORPORATION,5000000.00,Business,"Pilgrim's Pride Corporation, a major poultry p...",https://www.notus.org/donald-trump/inauguratio...
652,"RIPPLE LABS, INC.",4889345.33,Other,"Ripple Labs, Inc. has a connection to Donald T...",https://abcnews.go.com/US/sec-drops-case-crypt...


## Limitations

There are lots of things we can improve about this process. Here are some ideas:

- **Play around with the prompt**. We used a simple question, but the more specific you are, the better the results.
- **Use different search engines**. DuckDuckGo is free and good for a prototype. However, their API isn't meant to be used this way and will often return errors. It also doesn't return the best results. I recomment switching to Google, potentially via a SERP API.
- **Try other models**. If you find that gpt-4.1-mini is insufficient, you can use the smarter gpt-4.1 or a "reasoning" model like o3. Note that they are more expensive and slower.
- **Validate the output**. While we did code the classifier to return structured output, you can go beyond that and validate the output (ie. check that the sources are valid URLs).
- **Cache things**. Don't start over if something goes wrong, save the search results, scrapes and LLM outputs and continue where you left off.
- **Async/Multithreading**. This can be a pretty slow process, so you might want to run several agents in parallel.
- **Verify**. LLMs are still dumb and shouldn't be trusted! Manually verify the classifications if you're going to publish the results.

# A better way

I've used this approach for several projects at Global Witness, including our [annual classification of fossil fuel donors at COP](https://globalwitness.org/en/campaigns/fossil-fuels/how-we-used-ai-to-help-us-find-lobbyists-at-cop/).

To make my life easier, I packaged everything into a Python library called [Augmenta](https://github.com/Global-Witness/augmenta/). Let's see how it works.

First, we need to install Augmenta.

In [33]:
!uv pip install git+https://github.com/Global-Witness/augmenta.git

Using Python 3.13.2 environment at: D:\Projects\ddj-wiki\.venv
   Updating https://github.com/Global-Witness/augmenta.git (HEAD)
    Updated https://github.com/Global-Witness/augmenta.git (069913f704b4e8ed285db0d190abbf137c0e7888)
Resolved 102 packages in 5.36s
   Building augmenta @ git+https://github.com/Global-Witness/augmenta.git@069913f704b4e8ed285db0d190abbf137c0e7888
      Built augmenta @ git+https://github.com/Global-Witness/augmenta.git@069913f704b4e8ed285db0d190abbf137c0e7888
Prepared 10 packages in 3.65s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 14 packages in 321ms
 + aiolimiter==1.2.1
 + augmenta==0.1 (from git+https://github.com/Global-Witness/augmenta.git@069913f704b4e8ed285db0d190abbf137c0e7888)
 + googleapis-common-protos==1.70.0
 + logfire==3.16.0
 + opentelemetry-exporter-otlp

Next, we need to create a configuration file. This is a YAML file that contains all the settings for the agent, including the search engine to use, the LLM to use and the prompts. It may look something like this.

In [ ]:
yaml_config = """
input_csv: ai/data/fec_unclassified.csv
output_csv: ai/data/fec_classified.csv
model:
  provider: openai
  name: gpt-4.1-mini
search:
  engine: brightdata_google
prompt:
  system: You are an expert researcher whose job is to research the connections between political donors and Donald Trump.
  user: |
    # Instructions

    Your job is to research {{Name}}, a donor to Donald Trump's inaugural fund. Your will determine how Donald Trump and {{Name}} are connected, whether via family, business ties, political ties or something else. Don't consider donations to the inaugural fund as a connection, we're only interested in connections before or after the donation.

    ## Searching guidelines

    In most cases, you should start by searching for "{{Name}}" and "Donald Trump". Where relevant, remove redundant words like "INC", "LLC", "GROUP", etc from the search query. If you need to perform another search, try to refine it by adding relevant keywords. Note that each case will be different, so be flexible and adaptable. Unless necessary, limit your research to two or three searches.

    With each search, select a few sources that are most likely to provide relevant information. Access them using the tools provided. Be critical and use common sense. ALWAYS cite your sources.
    
    Now, please proceed with your analysis and classification of {{Name}}'s connections to Donald Trump.
structure:
  donor_industry:
    type: str
    description: What industry does this company, individual or organisation belong to?
  type_of_connection:
    type: str
    description: How are they connected to Donald Trump?
    options:
      - Family
      - Business
      - Political
      - Other
      - No connection
      - Don't know or not enough information
  explanation:
    type: str
    description: A few paragraphs explaining your decision in English, formatted in Markdown. In the explanation, link to the most relevant sources from the provided documents. Include at least one inline URL.
examples:
  - input: "Melinda Hildebrand"
    output:
      donor_industry: Energy
      type_of_connection: Political
      explanation: |
        President Donald Trump nominated River Oaks Donuts owner and energy executive Melinda Hildebrand [to serve as U.S. Ambassador to Costa Rica](https://www.congress.gov/nomination/119th-congress/55/21). In May 2024, [the Financial Times reported](https://www.ft.com/content/c89bbfc4-80db-4f3f-8d63-3aeb46ae91a9) the Hildebrand and her husband hosted a campaign event for then-presidential candidate Trump, who was seeking donations from power players in the U.S. energy industry. [The Daily Beast reported](https://www.thedailybeast.com/trump-goes-small-at-fundraisers-as-harris-goes-big-all-over/) that the Hildebrands also served on the host committee of a Trump fundraising dinner in Colorado. According to the Daily Beast, the host committee gave or raised $500,000 per couple.
logfire: true
"""

# Write the string directly to the file
with open("data/augmenta.yaml", 'w') as file:
    file.write(yaml_config)

Now, we just to run Augmenta. Because it's a CLI tool, it doesn't work well in a Jupyter notebook, so we'll run it in a terminal. You can run it like this:

```bash
augmenta data/augmenta.yaml -v
```

This approach is faster (parallel processing), more reliable (caching), more transparent (logging via logfire) and easier to use (no need to write code). It also allows you to use the same configuration file for different projects, so you can reuse your work.

You can also [extend Augmenta with custom tools](https://github.com/Global-Witness/augmenta/blob/main/docs/tools.md) via MCP servers. That means you can make it reason before each step, give it access to databases and APIs, and even allow it to write its own Python code (useful for processing data, doing maths, etc.).

Feel free to check the [README](https://github.com/Global-Witness/augmenta) and get in touch if you have any questions or suggestions.